In [1]:
# import sys
# sys.path.append('..')

In [2]:
from src.database.database_extract import RequestLocalAPI
extract = RequestLocalAPI()

/Users/M278428/opt/anaconda3/envs/test_dev_vrs_installation/lib/python3.9/site-packages/python_jsonschema_objects/__init__.py:49: UserWarning: Schema version http://json-schema.org/draft-07/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


ImportError: cannot import name 'RequestLocalAPI' from 'src.database.database_extract' (/Users/M278428/Documents/rf_lab_projects/DraftCoreDataModel/src/database/database_extract.py)

#### Extracting examples based on rowID

In [ ]:
example = extract.getExample(rowId=1)
example

#### Extracting all the examples in the DemoDatabase

In [ ]:
example_all = extract.getAllExamples()
example_all

#### Converting a row from the data from DemoCoreLiteDB into a core variant class

In [ ]:
example_cvc = extract.db_to_cvc(rowId=1)
example_cvc